# PyWry Plotly Integration

This notebook demonstrates some of PyWry's Plotly integration features, including:

1. **Custom Modebar Buttons** - Add theme toggle, log scale switch, and export buttons
2. **PyWry Toolbar** - Use dropdowns and buttons to control the chart and state from Python
3. **Bidirectional Communication** - Update figures from Python callbacks triggered by UI events

PyWry bundles Plotly.js 3.3.1 and provides Pydantic models for type-safe configuration.

## 1. Import Required Libraries

In [1]:
import numpy as np
import plotly.graph_objects as go

from pywry import (
    ModeBarButton,
    PyWry,
    PlotlyConfig,
    SvgIcon,
    ThemeMode,
)
from pywry.plotly_config import StandardButton
from pywry.toolbar import Toolbar, Button, Select, Option


In [2]:
# Create PyWry instance
# Automatically determines if running in Jupyter
app = PyWry(theme=ThemeMode.DARK, title="Plotly Integration Demo", width=1200, height=800)

## 2. Create Sample Data

Generate sample financial data with exponential growth (good for demonstrating log scale).

In [3]:
# Generate sample financial data with multiple series and complex structure
np.random.seed(42)
dates = np.arange("2020-01", "2025-01", dtype="datetime64[M]")
n = len(dates)

# Generate OHLC-style data for candlestick charts
base_price = 100 * np.exp(np.linspace(0, 1.5, n))
volatility = base_price * 0.05

# Multiple data series with different structures
data = {
    "dates": dates,
    # Standard time series
    "revenue": 100 * np.exp(np.linspace(0, 2, n)) + np.random.randn(n) * 20,
    "users": 1000 * np.exp(np.linspace(0, 3, n)) + np.random.randn(n) * 500,
    # OHLC data (tests open, high, low, close columns)
    "open": base_price + np.random.randn(n) * volatility,
    "high": base_price + np.abs(np.random.randn(n)) * volatility * 2,
    "low": base_price - np.abs(np.random.randn(n)) * volatility * 2,
    "close": base_price + np.random.randn(n) * volatility,
    # Text annotations (tests text column with special chars)
    "labels": [f"Q{(i % 4) + 1} '{20 + i // 12}" for i in range(n)],
    "notes": [f"Note: Value={v:.1f}, Change={np.random.randn():.2%}" for v in base_price],
    # Custom metadata (tests customdata with mixed types)
    "metadata": [{"quarter": (i % 4) + 1, "year": 2020 + i // 12, "flag": i % 3 == 0} for i in range(n)],
    # Costs breakdown by category (for pie chart)
    "costs": {
        "Salaries": 450000,
        "Marketing": 120000,
        "R&D": 180000,
        "Operations": 95000,
        "Infrastructure": 75000,
        "Legal & Compliance": 45000,
        "Other": 35000,
    },
}

print(f"Data range: {dates[0]} to {dates[-1]} ({n} points)")
print(f"Revenue range: ${data['revenue'].min():.0f} - ${data['revenue'].max():.0f}")
print(f"Users range: {data['users'].min():.0f} - {data['users'].max():.0f}")
print(f"OHLC range: ${data['low'].min():.0f} - ${data['high'].max():.0f}")
print(f"Costs breakdown: {len(data['costs'])} categories, total ${sum(data['costs'].values()):,}")

Data range: 2020-01 to 2024-12 (60 points)
Revenue range: $101 - $758
Users range: 554 - 20462
OHLC range: $92 - $499
Costs breakdown: 7 categories, total $1,000,000


## 3. Custom Modebar Buttons

PyWry provides `ModeBarButton` for creating custom buttons that emit PyWry events. This enables Python callbacks when users click modebar buttons.

In [4]:
# Create Custom SVG icons for buttons

theme_icon = SvgIcon(
    width=100,
    height=100,
    path="M50 30a20 20 0 1 0 0 40 20 20 0 0 0 0-40zM50 5l5 15h-10zM50 80l-5 15h10zM5 50l15-5v10zM80 50l15 5v-10zM18 18l12 8-5 5zM82 18l-8 12-5-5zM18 82l8-12 5 5zM82 82l-12-8 5-5z",
    transform="matrix(1 0 0 1 0 0)",
)
log_icon = SvgIcon(
    width=100,
    height=100,
    path="M10 90V10h5v75h75v5H10zm15-5V60h10v25H25zm15 0V40h10v45H40zm15 0V25h10v60H55zm15 0V15h10v70H70z",
    transform="matrix(1 0 0 1 0 0)",
)

download_icon = SvgIcon(
    width=100,
    height=100,
    path="M45 10h10v35h15L50 70 30 45h15V10zM15 75h70v15H15z",
    transform="matrix(1 0 0 1 0 0)",
)

# Create custom buttons
theme_toggle = ModeBarButton(
    name="toggleTheme",
    title="Toggle Dark/Light Theme",
    icon=theme_icon,
    event="plotly:toggle-theme",
)

# Log scale toggle button
log_scale_toggle = ModeBarButton(
    name="toggleLogScale",
    title="Toggle Log Scale",
    icon=log_icon,
    event="plotly:toggle-log-scale",
)

# Export data button
export_button = ModeBarButton(
    name="exportData",
    title="Export Data as CSV",
    icon=download_icon,
    event="plotly:export-data",
    data={"format": "csv"},
)

## 4. Configure PlotlyConfig

Use `PlotlyConfig` to customize the modebar, and any other desired configuration item.

In [5]:
# Create PlotlyConfig with custom buttons
# Remove ALL default buttons, keep only our custom ones
plotly_config = PlotlyConfig(
    display_mode_bar="hover",
    display_logo=False,
    scroll_zoom=True,
    mode_bar_buttons_to_remove=[
        *StandardButton
    ],
    mode_bar_buttons_to_add=[
        log_scale_toggle,
        export_button,
        theme_toggle,
    ],
)

## 5. Create PyWry Toolbar

Add a toolbar with a dropdowns to select different data series. In this example, the second dropdown is controlled by the first. The button resets to the original view.

In [6]:
# Create toolbar with chart controls
# Data = what data to show, Type = how to visualize it
toolbar = Toolbar(
    position="top",
    items=[
        Select(
            label="Data:",
            event="chart:select_series", # Events can be any custom:event_name
            options=[
                Option(label="Stock Price", value="stock"),  # OHLC data
                Option(label="Revenue", value="revenue"),
                Option(label="Users", value="users"),
                Option(label="Costs", value="costs"),
            ],
            selected="stock",
        ),
        Select(
            component_id="chart-type-select",  # Fixed ID so we can update options dynamically
            label="Type:",
            event="chart:select_type",  # Event name will map to the on_select_type handler
            options=[
                Option(label="Candlestick", value="candlestick"),  # Only for stock
                Option(label="Line", value="line"),
                Option(label="Bar", value="bar"),
                Option(label="Area", value="area"),
            ],
            selected="candlestick",
        ),
        Button(
            label="Reset",
            event="chart:reset",
        ),
    ],
)

## 6. Define Event Handlers

Create Python callbacks that respond to toolbar and modebar events. These update the chart in real-time, and we can manage the state from Python.

In [ ]:
# Track current state
chart_state = {
    "series": "stock",       # stock, revenue, users, costs
    "chart_type": "candlestick",  # candlestick, line, bar, area, pie
    "is_dark_theme": True,
    "is_log_scale": False,
}

# Track chart type per data series (remembers selection when switching back)
chart_type_per_series = {
    "stock": "candlestick",
    "revenue": "line",
    "users": "line",
    "costs": "pie",
}

widget = None
config_dict = None

# Chart type options based on data series
STOCK_CHART_TYPES = [
    {"label": "Candlestick", "value": "candlestick"},
    {"label": "Line", "value": "line"},
    {"label": "Bar", "value": "bar"},
    {"label": "Area", "value": "area"},
]

OTHER_CHART_TYPES = [
    {"label": "Line", "value": "line"},
    {"label": "Bar", "value": "bar"},
    {"label": "Area", "value": "area"},
]

COSTS_CHART_TYPES = [
    {"label": "Pie", "value": "pie"},
]


def build_figure():
    """Build Plotly figure based on current state.
    
    - Stock data supports: candlestick, line, bar, area
    - Revenue/Users: line, bar, area
    - Costs: pie only (breakdown by category)
    """
    series = chart_state["series"]
    chart_type = chart_state["chart_type"]
    is_dark = chart_state["is_dark_theme"]
    is_log = chart_state["is_log_scale"]
    
    x_data = data["dates"]
    template = "plotly_dark" if is_dark else "plotly_white"
    
    # Determine what data and how to show it
    if series == "stock":
        # Stock price data - can use candlestick or standard charts
        if chart_type == "candlestick":
            fig = go.Figure(data=[
                go.Candlestick(x=x_data, open=data["open"], high=data["high"],
                               low=data["low"], close=data["close"], name="Stock")
            ])
        elif chart_type == "line":
            fig = go.Figure(data=[
                go.Scatter(x=x_data, y=data["close"], mode="lines+markers", name="Close Price")
            ])
        elif chart_type == "bar":
            fig = go.Figure(data=[
                go.Bar(x=x_data, y=data["close"], name="Close Price")
            ])
            fig.update_xaxes(showticklabels=False)  # Hide x-axis labels for bar
        elif chart_type == "area":
            fig = go.Figure(data=[
                go.Scatter(x=x_data, y=data["close"], fill="tozeroy", name="Close Price")
            ])
        else:
            fig = go.Figure(data=[go.Scatter(x=x_data, y=data["close"], name="Close")])
        title = "Stock Price"
        y_title = "Price ($)"
    elif series == "costs":
        # Costs - pie chart by category
        costs = data["costs"]
        fig = go.Figure(data=[
            go.Pie(
                labels=list(costs.keys()),
                values=list(costs.values()),
                name="Costs",
                textinfo="label+percent",
                hovertemplate="<b>%{label}</b><br>$%{value:,.0f}<br>%{percent}<extra></extra>",
            )
        ])
        title = "Cost Breakdown by Category"
        y_title = ""
    else:
        # Other data series (revenue, users) - line, bar, area
        y_data = data[series]
        chart_type = chart_type if chart_type in ["line", "bar", "area"] else "line"
        
        if chart_type == "line":
            trace = go.Scatter(x=x_data, y=y_data, mode="lines+markers", name=series.title(),
                               text=data["labels"], customdata=data["notes"])
            fig = go.Figure(data=[trace])
        elif chart_type == "bar":
            trace = go.Bar(x=x_data, y=y_data, name=series.title())
            fig = go.Figure(data=[trace])
            fig.update_xaxes(showticklabels=False)  # Hide x-axis labels for bar
        elif chart_type == "area":
            trace = go.Scatter(x=x_data, y=y_data, fill="tozeroy", name=series.title(),
                               text=data["labels"])
            fig = go.Figure(data=[trace])
        else:
            trace = go.Scatter(x=x_data, y=y_data, mode="lines", name=series.title())
            fig = go.Figure(data=[trace])
        
        title = f"{series.title()} Over Time"
        y_title = series.title()
    
    # Log scale (not for candlestick or pie)
    y_type = "log" if is_log and chart_type not in ["candlestick", "pie"] else "linear"
    
    layout_update = {
        "template": template,
        "title": title,
        "margin": dict(t=60, b=40, l=60, r=40),
    }
    
    # Only add axis titles for non-pie charts
    if chart_type != "pie":
        layout_update["xaxis_title"] = "Date"
        layout_update["yaxis_title"] = y_title
        layout_update["yaxis_type"] = y_type
    
    fig.update_layout(**layout_update)
    
    return fig


def on_select_series(event_data, event_type, label):
    """Handle data series selection - restore saved chart type for this series."""
    new_series = event_data.get("value", "stock")
    chart_state["series"] = new_series
    
    # Restore the saved chart type for this series
    saved_chart_type = chart_type_per_series.get(new_series, "line")
    chart_state["chart_type"] = saved_chart_type
    
    # Get options for this series
    if new_series == "stock":
        options = STOCK_CHART_TYPES
    elif new_series == "costs":
        options = COSTS_CHART_TYPES
    else:
        options = OTHER_CHART_TYPES
    
    # Update Type dropdown options via toolbar:set-value
    widget.emit("toolbar:set-value", {
        "componentId": "chart-type-select",
        "options": options,
        "value": saved_chart_type,
    })
    
    # Update figure
    fig = build_figure()
    widget.update_figure(fig, config=config_dict)
    print(f"Data: {chart_state['series']}, Type: {chart_state['chart_type']} (restored)")


def on_select_type(event_data, event_type, label):
    """Handle chart type selection - save for current series."""
    new_type = event_data.get("value", "line")
    chart_state["chart_type"] = new_type
    
    # Save chart type for this series
    chart_type_per_series[chart_state["series"]] = new_type
    
    fig = build_figure()
    widget.update_figure(fig, config=config_dict)
    print(f"Data: {chart_state['series']}, Type: {chart_state['chart_type']} (saved)")


def on_toggle_theme(event_data, event_type, label):
    """Toggle dark/light theme."""
    chart_state["is_dark_theme"] = not chart_state["is_dark_theme"]
    new_theme = "dark" if chart_state["is_dark_theme"] else "light"
    widget.theme = new_theme
    widget.emit("pywry:update-theme", {"theme": new_theme})
    print(f"Theme: {new_theme}")


def on_toggle_log_scale(event_data, event_type, label):
    """Toggle log/linear scale."""
    chart_state["is_log_scale"] = not chart_state["is_log_scale"]
    scale = "log" if chart_state["is_log_scale"] else "linear"
    widget.update_layout({"yaxis.type": scale})
    print(f"Scale: {scale}")


def on_export_data(event_data, event_type, label):
    """Export current chart data as CSV - send to JS for download."""
    import io
    import csv
    
    series = chart_state["series"]
    chart_type = chart_state["chart_type"]
    
    # Build CSV based on current data
    output = io.StringIO()
    writer = csv.writer(output)
    
    if series == "stock":
        if chart_type == "candlestick":
            writer.writerow(["date", "open", "high", "low", "close"])
            for i, date in enumerate(data["dates"]):
                writer.writerow([date, data["open"][i], data["high"][i], data["low"][i], data["close"][i]])
        else:
            writer.writerow(["date", "close"])
            for i, date in enumerate(data["dates"]):
                writer.writerow([date, data["close"][i]])
        filename = "stock_price.csv"
    elif series == "costs":
        # Export cost breakdown by category
        writer.writerow(["category", "amount"])
        for category, amount in data["costs"].items():
            writer.writerow([category, amount])
        filename = "costs_breakdown.csv"
    else:
        writer.writerow(["date", series, "label"])
        for i, date in enumerate(data["dates"]):
            writer.writerow([date, data[series][i], data["labels"][i]])
        filename = f"{series}_over_time.csv"
    
    csv_content = output.getvalue()
    output.close()
    
    # Send CSV to JavaScript for download
    widget.emit("pywry:download-csv", {"csv": csv_content, "filename": filename})


def on_reset(event_data, event_type, label):
    """Reset to initial state."""
    chart_state.update({"series": "stock", "chart_type": "candlestick", "is_dark_theme": True, "is_log_scale": False})
    
    # Reset saved chart types to defaults
    chart_type_per_series.update({
        "stock": "candlestick",
        "revenue": "line",
        "users": "line",
        "costs": "pie",
    })
    
    # Reset Type dropdown to stock options
    widget.emit("toolbar:set-value", {
        "componentId": "chart-type-select",
        "options": STOCK_CHART_TYPES,
        "value": "candlestick",
    })
    
    fig = build_figure()
    widget.update_figure(fig, config=config_dict)
    widget.theme = "dark"
    widget.emit("pywry:update-theme", {"theme": "dark"})
    print("Reset to: Stock + Candlestick + Dark")


print("Callbacks ready. Chart type is remembered per data series:")
print("  Stock: Candlestick (default), Line, Bar, Area")
print("  Revenue/Users: Line (default), Bar, Area")
print("  Costs: Pie (only option)")

Callbacks ready. Chart type is remembered per data series:
  Stock: Candlestick (default), Line, Bar, Area
  Revenue/Users: Line (default), Bar, Area
  Costs: Pie (only option)


## 7. Display the Interactive Chart

Show the Plotly figure with custom modebar and toolbar. The `callbacks` parameter connects events to Python handlers.

In [8]:
# Full test with custom config + toolbar + callbacks
fig = build_figure()

# Store config dict for use in callbacks (to preserve modebar buttons on update)
config_dict = plotly_config.model_dump(by_alias=True, exclude_none=True)

widget = app.show_plotly(
    fig,
    config=plotly_config,
    toolbars=[toolbar],
    callbacks={
        # Toolbar events (user-defined 'chart' namespace)
        "chart:select_series": on_select_series,
        "chart:select_type": on_select_type,
        "chart:reset": on_reset,
        # Modebar events (plotly namespace)
        "plotly:toggle-theme": on_toggle_theme,
        "plotly:toggle-log-scale": on_toggle_log_scale,
        "plotly:export-data": on_export_data,
    },
)

## 8. Summary

This notebook demonstrated:

### Custom Modebar Buttons
- `ModeBarButton` with `event` parameter emits PyWry events on click
- `SvgIcon` for custom button icons
- `PlotlyIconName` enum for built-in Plotly icons
- `PlotlyConfig.mode_bar_buttons_to_add` and `mode_bar_buttons_to_remove`

### PyWry Toolbar
- `Select` dropdown triggers `chart:select_series` and `chart:select_type` events
- `Button` triggers `chart:reset` event
- All toolbar events call Python handlers

### Figure Updates via Widget Methods
The widget returned by `show_plotly()` provides clean Python methods:

| Method | Use Case |
|--------|----------|
| `widget.update_figure(fig)` | Replace entire chart (data + layout) |
| `widget.update_layout(props)` | Update layout properties only |
| `widget.update_traces(patch)` | Update trace properties (restyle) |

### Event Flow
```
User clicks button → PyWry event emitted → Python callback triggered →
callback calls widget.update_figure() → widget emits plotly:update-figure →
JavaScript Plotly.react() updates chart
```

No manual JavaScript required!